__PHARML 2022 CHALLENGE - SURVIVAL ANALYSIS MODELING SECTION

Prognostic models have a wide-array of applications and are used in academic and clinical settings. Lung cancer is newly diagnosed in 2.3 million people per year and is divided in Non-Small Cell Lung Cancer (NSCLC) and Small Cell Lung Cancer (SCLC). NSCLC accounts for 84% of the total cases. The 5-year survival rate for patients with NSCLC is 25%, but prognosis can vary depending on many factors including demographics, clinical characteristics, and genomic mutations, among others. Survival Machine Learning models could enable us to make much better prognosis prediction, which in turn has real world clinical applications. Prognostic models could be used to help us identify the most predictive and novel biomarkers influencing the risk of mortality in NSCLC. Additionally, representations learned by fitting Survival Machine Learning models on NSCLC data could be used to stratify patients and obtain clinical clusters or phenotypes that give us insights on disease heterogeneity. Yet another promising application for these models is the potential use of covariate adjustment methods to increase the power of clinical trials and achieve significant results with fewer patients. Finally, it should be noted that there are many other potential relevant real-world applications for these methods.



In [1]:
#Import Library
library(dplyr)
library(ggplot2)
library(tidyr)
library(caret)
library(lubridate)
library(purrr)
library(mgcv)
library(rpart)
library(tidyverse)
library(utils)
library(broom)
library(ellipsis)
library(highr)
library(lattice)
library(methods)
library(nlme)
library(readr)
library(readxl)
library(stats)
library(stringr)
library(ranger)
library(tibble)
library(xgboost)
library(magrittr)
library(lubridate)
library(xts)
library(hms)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: lattice

Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”

Attaching package: ‘lubridate’


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union



Attaching package: ‘purrr’


The following object is masked from ‘package:caret’:

    lift


Warning message:
“package ‘mgcv’ was built under R version 4.1.3”
Loading required package: nlme


Attaching package: ‘nlme’


The following object is masked from ‘package:dplyr’:

    collapse


This is mgcv 1.8-40. For overview type 'help("mgcv-package")'.

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ tibble  3.1.6     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────

In [2]:
flatiron_train <- read_csv("/pharml/data/flatiron_training.csv")
head(flatiron_train)

Rows: 31334 Columns: 25
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (12): patientid, ecog, histology, groupstage, smokingstatus, gender, rac...
dbl  (7): daysto_event, event_died, diagnosis_nsclc_year, diagnosis_ansclc_y...
lgl  (6): biomarker_pdl1, biomarker_alk, biomarker_ros1, biomarker_braf, bio...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


patientid,daysto_event,event_died,diagnosis_nsclc_year,diagnosis_ansclc_year,ecog,histology,groupstage,smokingstatus,birthyear,⋯,biomarker_alk,biomarker_ros1,biomarker_braf,biomarker_egfr,biomarker_kras,lab_creatinine,lab_alanineaminotransferase,lab_ldh,bmi,index_lot1_year
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,⋯,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<chr>,<chr>,<chr>,<dbl>,<dbl>
PHARML000001,156,0,2002,2019,1,Squamous cell carcinoma,Stage IIA,History of smoking,1951,⋯,NA,NA,NA,NA,NA,normal,normal,NA,19.34262,2019
PHARML000003,190,1,2013,2014,Unknown,Non-squamous cell carcinoma,Stage IIIA,History of smoking,1946,⋯,NA,NA,NA,NA,NA,NA,normal,NA,22.41903,2014
PHARML000004,74,1,2020,2020,Unknown,Non-squamous cell carcinoma,Stage IIIC,History of smoking,1944,⋯,NA,NA,NA,NA,NA,NA,NA,NA,26.63880,2020
PHARML000008,52,1,2020,2020,1,Non-squamous cell carcinoma,Stage IV,History of smoking,1937,⋯,NA,NA,NA,NA,NA,normal,abnormal,NA,18.94848,2020
PHARML000009,392,0,2021,2021,0,Non-squamous cell carcinoma,Stage IIIB,History of smoking,1966,⋯,NA,NA,NA,NA,NA,normal,normal,abnormal,21.33351,2021
PHARML000011,39,1,2017,2017,Unknown,Non-squamous cell carcinoma,Stage IV,History of smoking,1943,⋯,NA,NA,NA,NA,NA,normal,normal,normal,16.65117,2017


In [3]:
flatiron_train1 <- flatiron_train %>% filter(!is.na(diagnosis_nsclc_year))
summary(flatiron_train1)

  patientid          daysto_event      event_died     diagnosis_nsclc_year
 Length:31329       Min.   :   1.0   Min.   :0.0000   Min.   :1990        
 Class :character   1st Qu.: 112.0   1st Qu.:0.0000   1st Qu.:2014        
 Mode  :character   Median : 280.0   Median :1.0000   Median :2016        
                    Mean   : 484.2   Mean   :0.7273   Mean   :2016        
                    3rd Qu.: 639.0   3rd Qu.:1.0000   3rd Qu.:2019        
                    Max.   :3948.0   Max.   :1.0000   Max.   :2022        
                                                                          
 diagnosis_ansclc_year     ecog            histology          groupstage       
 Min.   :2011          Length:31329       Length:31329       Length:31329      
 1st Qu.:2014          Class :character   Class :character   Class :character  
 Median :2017          Mode  :character   Mode  :character   Mode  :character  
 Mean   :2016                                                                  


In [5]:
library("naniar", lib.loc=.libPaths("/tmp/RtmpopXkDN/downloaded_packages"))

In [6]:
flatiron_train1 <- flatiron_train1 %>% replace_with_na(replace = list(bmi = "<NA>"), 
                                                       (lot_category = "<NA>"))

In [7]:
toReplace1 <- which(is.na(flatiron_train1$bmi))
flatiron_train1$bmi[toReplace1] <- 19.0

In [8]:
library("superml", lib.loc=.libPaths("/tmp/RtmpopXkDN/downloaded_packages"))

Loading required package: R6



In [9]:
label <- LabelEncoder$new()
print(label$fit(flatiron_train1$ecog))
flatiron_train1$ecog <- label$fit_transform(flatiron_train1$ecog)

label <- LabelEncoder$new()
print(label$fit(flatiron_train1$groupstage))
flatiron_train1$groupstage <- label$fit_transform(flatiron_train1$groupstage)

label <- LabelEncoder$new()
print(label$fit(flatiron_train1$smokingstatus))
flatiron_train1$smokingstatus <- label$fit_transform(flatiron_train1$smokingstatus)

label <- LabelEncoder$new()
print(label$fit(flatiron_train1$gender))
flatiron_train1$gender <- label$fit_transform(flatiron_train1$gender)

label <- LabelEncoder$new()
print(label$fit(flatiron_train1$lab_ldh))
flatiron_train1$lab_ldh <- label$fit_transform(flatiron_train1$lab_ldh)

label <- LabelEncoder$new()
print(label$fit(flatiron_train1$lab_alanineaminotransferase))
flatiron_train1$lab_alanineaminotransferase <- label$fit_transform(flatiron_train1$lab_alanineaminotransferase)

label <- LabelEncoder$new()
print(label$fit(flatiron_train1$lab_creatinine))
flatiron_train1$lab_creatinine <- label$fit_transform(flatiron_train1$lab_creatinine)

label <- LabelEncoder$new()
print(label$fit(flatiron_train1$biomarker_pdl1))
flatiron_train1$biomarker_pdl1 <- label$fit_transform(flatiron_train1$biomarker_pdl1)

label <- LabelEncoder$new()
print(label$fit(flatiron_train1$biomarker_alk))
flatiron_train1$biomarker_alk <- label$fit_transform(flatiron_train1$biomarker_alk)

label <- LabelEncoder$new()
print(label$fit(flatiron_train1$biomarker_ros1))
flatiron_train1$biomarker_ros1 <- label$fit_transform(flatiron_train1$biomarker_ros1)

label <- LabelEncoder$new()
print(label$fit(flatiron_train1$biomarker_braf))
flatiron_train1$biomarker_braf <- label$fit_transform(flatiron_train1$biomarker_braf)

label <- LabelEncoder$new()
print(label$fit(flatiron_train1$biomarker_egfr))
flatiron_train1$biomarker_egfr <- label$fit_transform(flatiron_train1$biomarker_egfr)

label <- LabelEncoder$new()
print(label$fit(flatiron_train1$biomarker_kras))
flatiron_train1$biomarker_kras <- label$fit_transform(flatiron_train1$biomarker_kras)

[1] TRUE
[1] TRUE
[1] TRUE


The data contains NA values. Imputing NA with 'NA' 



[1] TRUE


The data contains NA values. Imputing NA with 'NA' 

The data contains NA values. Imputing NA with 'NA' 



[1] TRUE


...

The data contains NA values. Imputing NA with 'NA' 

The data contains NA values. Imputing NA with 'NA' 



[1] TRUE


The data contains NA values. Imputing NA with 'NA' 

The data contains NA values. Imputing NA with 'NA' 



[1] TRUE


The data contains NA values. Imputing NA with 'NA' 



In [10]:
flatiron_train2 <- flatiron_train1 %>% mutate(ageAtdiagnosis = diagnosis_nsclc_year - birthyear, 
    yrsToprogress = diagnosis_ansclc_year - diagnosis_nsclc_year, 
                                              TimetoTx = index_lot1_year - diagnosis_nsclc_year, 
                                            TxtoFail = diagnosis_ansclc_year - index_lot1_year)
head(flatiron_train2)

patientid,daysto_event,event_died,diagnosis_nsclc_year,diagnosis_ansclc_year,ecog,histology,groupstage,smokingstatus,birthyear,⋯,biomarker_kras,lab_creatinine,lab_alanineaminotransferase,lab_ldh,bmi,index_lot1_year,ageAtdiagnosis,yrsToprogress,TimetoTx,TxtoFail
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
PHARML000001,156,0,2002,2019,0,Squamous cell carcinoma,0,0,1951,⋯,0,0,0,0,19.34262,2019,51,17,17,0
PHARML000003,190,1,2013,2014,1,Non-squamous cell carcinoma,1,0,1946,⋯,0,1,0,0,22.41903,2014,67,1,1,0
PHARML000004,74,1,2020,2020,1,Non-squamous cell carcinoma,2,0,1944,⋯,0,1,1,0,26.63880,2020,76,0,0,0
PHARML000008,52,1,2020,2020,0,Non-squamous cell carcinoma,3,0,1937,⋯,0,0,2,0,18.94848,2020,83,0,0,0
PHARML000009,392,0,2021,2021,2,Non-squamous cell carcinoma,4,0,1966,⋯,0,0,0,1,21.33351,2021,55,0,0,0
PHARML000011,39,1,2017,2017,1,Non-squamous cell carcinoma,3,0,1943,⋯,0,0,0,2,16.65117,2017,74,0,0,0


In [11]:
flatiron_train3 <- flatiron_train2 %>% 
                        mutate(squam_cells = ifelse(histology == "Squamous cell carcinoma", 1, 0), 
                        non_squamCells = ifelse(histology == "Non-squamous cell carcinoma", 1, 0), 
                        NSCLC = ifelse(histology == "NSCLC histology NOS", 1, 0), 
                        Hisp_Latino = ifelse(raceethnicity == "Hispanic or Latino (any)", 1, 0), 
                        White = ifelse(raceethnicity == "Non-Hispanic White", 1, 0), 
                        Black = ifelse(raceethnicity == "Non-Hispanic Black", 1, 0), 
                        Asian = ifelse(raceethnicity == "Non-Hispanic Asian", 1, 0), 
                        Other_Race = ifelse(raceethnicity == "Other Race", 1, 0),
                        Midwest = ifelse(region == "Midwest", 1, 0),
                        Northeast = ifelse(region == "Northeast", 1, 0),
                        South = ifelse(region == "South", 1, 0),
                        West = ifelse(region == "West", 1, 0),
                        Other = ifelse(region == "0", 1, 0),
                        Chemotherapy = ifelse(lot_category == "Chemotherapy", 1, 0),
                        Immunotherapy = ifelse(lot_category == "Immunotherapy", 1, 0),
                        TargBiologic = ifelse(lot_category == "Targeted/Biologic", 1, 0),
                        TargNonBiologic = ifelse(lot_category == "Targeted/Non-Biologic", 1, 0),
                        Hormone = ifelse(lot_category == "Hormone", 1, 0),
                        AntibodyConj = ifelse(lot_category == "Antibody-Conjugate", 1, 0),
                        Cytoprotective = ifelse(lot_category == "Cytoprotective", 1, 0))
                               
head(flatiron_train2)

patientid,daysto_event,event_died,diagnosis_nsclc_year,diagnosis_ansclc_year,ecog,histology,groupstage,smokingstatus,birthyear,⋯,biomarker_kras,lab_creatinine,lab_alanineaminotransferase,lab_ldh,bmi,index_lot1_year,ageAtdiagnosis,yrsToprogress,TimetoTx,TxtoFail
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
PHARML000001,156,0,2002,2019,0,Squamous cell carcinoma,0,0,1951,⋯,0,0,0,0,19.34262,2019,51,17,17,0
PHARML000003,190,1,2013,2014,1,Non-squamous cell carcinoma,1,0,1946,⋯,0,1,0,0,22.41903,2014,67,1,1,0
PHARML000004,74,1,2020,2020,1,Non-squamous cell carcinoma,2,0,1944,⋯,0,1,1,0,26.63880,2020,76,0,0,0
PHARML000008,52,1,2020,2020,0,Non-squamous cell carcinoma,3,0,1937,⋯,0,0,2,0,18.94848,2020,83,0,0,0
PHARML000009,392,0,2021,2021,2,Non-squamous cell carcinoma,4,0,1966,⋯,0,0,0,1,21.33351,2021,55,0,0,0
PHARML000011,39,1,2017,2017,1,Non-squamous cell carcinoma,3,0,1943,⋯,0,0,0,2,16.65117,2017,74,0,0,0


In [12]:
flatiron_select <- flatiron_train3 %>% select(-diagnosis_nsclc_year, -diagnosis_ansclc_year, -histology, 
                            -raceethnicity, -region, -lot_category, -index_lot1_year, -Midwest, -South,
                               -Other_Race, -Northeast, -West, -Other, -lab_ldh, -birthyear, -TxtoFail,
                            -lab_alanineaminotransferase, -lab_creatinine, -patientid, -Cytoprotective, 
                            -Other, -Other_Race, -NSCLC)

In [13]:
library("survival", lib.loc=.libPaths("/tmp/RtmpopXkDN/downloaded_packages"))


Attaching package: ‘survival’


The following object is masked from ‘package:caret’:

    cluster




In [14]:
flatiron_select1 <- na.exclude(flatiron_select)

In [15]:
#5 fold cross validation
# Fit lm model using 5-fold CV: model


#set seed for model reproducibility
set.seed(42)
model <- train(
  daysto_event ~ ., 
  flatiron_select1,
  method = "lm",
  trControl = trainControl(
    method = "cv", 
    number = 5,
    verboseIter = TRUE
    
  )
)

# Print model to console
model

+ Fold1: intercept=TRUE 
- Fold1: intercept=TRUE 
+ Fold2: intercept=TRUE 
- Fold2: intercept=TRUE 
+ Fold3: intercept=TRUE 
- Fold3: intercept=TRUE 
+ Fold4: intercept=TRUE 
- Fold4: intercept=TRUE 
+ Fold5: intercept=TRUE 
- Fold5: intercept=TRUE 
Aggregating results
Fitting final model on full training set


Linear Regression 

30471 samples
   27 predictor

No pre-processing
Resampling: Cross-Validated (5 fold) 
Summary of sample sizes: 24377, 24376, 24377, 24377, 24377 
Resampling results:

  RMSE      Rsquared  MAE     
  528.5786  0.110594  372.1656

Tuning parameter 'intercept' was held constant at a value of TRUE

In [16]:
#Split datasets into Train/Predict sets
N_UK <- nrow(flatiron_select1)
N_UK

#Number of rows of training data required for model
target_UK <- round(N_UK * 0.75)
target_UK

#Creating vector of N_UK uniform variables
gp_UK <- runif(N_UK)

#Create training set flatiron_trainSet (75% of data) & flatiron_predictSet (25% of data)
flatiron_trainSet <- flatiron_select1[gp_UK < 0.75, ]
flatiron_validSet <- flatiron_select1[gp_UK >= 0.75, ]

#Examine flatiron_trainSet and flatiron_predictSet datasets
nrow(flatiron_trainSet)
nrow(flatiron_validSet)

[1] 30471

[1] 22853

[1] 22808

[1] 7663

In [17]:
library("rms", lib.loc=.libPaths("/tmp/RtmpopXkDN/downloaded_packages"))

Loading required package: Hmisc

Loading required package: Formula


Attaching package: ‘Hmisc’


The following objects are masked from ‘package:dplyr’:

    src, summarize


The following objects are masked from ‘package:base’:

    format.pval, units


Loading required package: SparseM


Attaching package: ‘SparseM’


The following object is masked from ‘package:base’:

    backsolve




In [21]:
#Fit a Multivariate COX PH regression analysis model
units(flatiron_trainSet$daysto_event) <- "Days"
dd <- datadist(flatiron_trainSet)
options(datadist = "dd")
fitCPH1 <- cph(Surv(daysto_event, event_died) ~ 
               biomarker_pdl1 + biomarker_egfr + biomarker_alk + biomarker_ros1  + ecog + bmi + 
               ageAtdiagnosis + biomarker_braf + biomarker_kras + yrsToprogress + TimetoTx + gender + 
               groupstage + smokingstatus + ecog + squam_cells + non_squamCells +  + Hisp_Latino + Black + 
               White + Asian + Chemotherapy + Immunotherapy + Hormone + TargBiologic + TargNonBiologic +
               AntibodyConj, 
               data = flatiron_trainSet, x = TRUE, y = TRUE, surv = TRUE, time.inc = 1) 

In [22]:
print(fitCPH1)

Cox Proportional Hazards Model
 
 cph(formula = Surv(daysto_event, event_died) ~ biomarker_pdl1 + 
     biomarker_egfr + biomarker_alk + biomarker_ros1 + ecog + 
     bmi + ageAtdiagnosis + biomarker_braf + biomarker_kras + 
     yrsToprogress + TimetoTx + gender + groupstage + smokingstatus + 
     ecog + squam_cells + non_squamCells + +Hisp_Latino + Black + 
     White + Asian + Chemotherapy + Immunotherapy + Hormone + 
     TargBiologic + TargNonBiologic + AntibodyConj, data = flatiron_trainSet, 
     x = TRUE, y = TRUE, surv = TRUE, time.inc = 1)
 
                          Model Tests       Discrimination    
                                                   Indexes    
 Obs      22808    LR chi2    1403.60       R2       0.060    
 Events   16582    d.f.            26    R2(26,22808)0.059    
 Center -0.2998    Pr(> chi2)  0.0000    R2(26,16582)0.080    
                   Score chi2 1377.56       Dxy      0.190    
                   Pr(> chi2)  0.0000                         


In [23]:
#Fit a Multivariate COX PH regression analysis model
units(flatiron_trainSet$daysto_event) <- "Days"
dd <- datadist(flatiron_trainSet)
options(datadist = "dd")
fitCPH2 <- cph(Surv(daysto_event, event_died) ~ 
               biomarker_pdl1 + biomarker_egfr + biomarker_alk + biomarker_ros1  + ecog + bmi + 
               ageAtdiagnosis + biomarker_braf + biomarker_kras + yrsToprogress + TimetoTx + gender + 
               groupstage + smokingstatus + ecog + squam_cells + non_squamCells +  + Hisp_Latino + Black + 
               White + Asian + Chemotherapy + Immunotherapy + Hormone + TargBiologic + TargNonBiologic +
               AntibodyConj + groupstage * bmi + smokingstatus * ecog + ageAtdiagnosis * smokingstatus +
               biomarker_egfr * groupstage, 
               data = flatiron_trainSet, x = TRUE, y = TRUE, surv = TRUE, time.inc = 1) 

In [24]:
print(fitCPH2)

Cox Proportional Hazards Model
 
 cph(formula = Surv(daysto_event, event_died) ~ biomarker_pdl1 + 
     biomarker_egfr + biomarker_alk + biomarker_ros1 + ecog + 
     bmi + ageAtdiagnosis + biomarker_braf + biomarker_kras + 
     yrsToprogress + TimetoTx + gender + groupstage + smokingstatus + 
     ecog + squam_cells + non_squamCells + +Hisp_Latino + Black + 
     White + Asian + Chemotherapy + Immunotherapy + Hormone + 
     TargBiologic + TargNonBiologic + AntibodyConj + groupstage * 
     bmi + smokingstatus * ecog + ageAtdiagnosis * smokingstatus + 
     biomarker_egfr * groupstage, data = flatiron_trainSet, x = TRUE, 
     y = TRUE, surv = TRUE, time.inc = 1)
 
                          Model Tests       Discrimination    
                                                   Indexes    
 Obs      22808    LR chi2    1418.19       R2       0.060    
 Events   16582    d.f.            30    R2(30,22808)0.059    
 Center -0.3797    Pr(> chi2)  0.0000    R2(30,16582)0.080    
         

In [25]:
#Validating the model

validate(fitCPH2, method = "crossvalidation", B = 10, pr = FALSE)

      index.orig training   test optimism index.corrected  n
Dxy       0.1912   0.1913 0.1883   0.0029          0.1882 10
R2        0.0603   0.0604 0.0586   0.0019          0.0584 10
Slope     1.0000   1.0000 0.9799   0.0201          0.9799 10
D         0.0046   0.0047 0.0060  -0.0013          0.0059 10
U         0.0000   0.0000 0.0001  -0.0001          0.0001 10
Q         0.0047   0.0047 0.0059  -0.0012          0.0058 10
g         0.3353   0.3357 0.3287   0.0069          0.3283 10

In [26]:
library("dynpred", "/tmp/RtmpopXkDN/downloaded_packages")

In [27]:
#Compute concordance index on validation set
cindex(Surv(daysto_event, event_died) ~ yrsToprogress + bmi, data = flatiron_validSet)

$concordant
[1] 12252270

$total
[1] 22637044

$cindex
[1] 0.5412487

In [28]:
library("pec", "/tmp/RtmpopXkDN/downloaded_packages")

Loading required package: prodlim


Attaching package: ‘pec’


The following object is masked from ‘package:dynpred’:

    cindex


The following object is masked from ‘package:caret’:

    R2




In [29]:
A1 <- pec::cindex(list("Cox X1"=fitCPH2),
formula=Surv(daysto_event, event_died) ~ ecog + groupstage + smokingstatus + 
               squam_cells + non_squamCells + biomarker_pdl1 + biomarker_egfr + biomarker_alk 
               + biomarker_braf + biomarker_ros1 + biomarker_kras + bmi + ageAtdiagnosis + yrsToprogress + 
                TimetoTx + gender + Hisp_Latino + Black + White + Asian,
data=flatiron_select1,
eval.times=1)

In [30]:
head(A1)

$AppCindex
$AppCindex$`Cox X1`
[1] 0.6930943


$Pairs
$Pairs$`Cox X1`
[1] 1219622


$Concordant
$Concordant$`Cox X1`
[1] 845312.8


$call
pec::cindex(object = list(`Cox X1` = fitCPH2), formula = Surv(daysto_event, 
    event_died) ~ ecog + groupstage + smokingstatus + squam_cells + 
    non_squamCells + biomarker_pdl1 + biomarker_egfr + biomarker_alk + 
    biomarker_braf + biomarker_ros1 + biomarker_kras + bmi + 
    ageAtdiagnosis + yrsToprogress + TimetoTx + gender + Hisp_Latino + 
    Black + White + Asian, data = flatiron_select1, eval.times = 1)

$time
[1] 1

$pred.time
[1] 1
